In [1]:
# Import library
import pandas as pd
import numpy as np
import os, random, warnings, gc, psutil, datetime
from tqdm import tqdm_notebook, tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from math import sqrt

import lightgbm as lgbm
from catboost import CatBoostRegressor

from glob import glob
from IPython.display import display

import seaborn as sns
import matplotlib.pyplot as plt

# Set options
pd.set_option('max_columns',500)
pd.set_option('max_rows',500)
pd.options.display.max_colwidth = 300

warnings.filterwarnings('ignore')

%matplotlib inline
sns.set_palette('bright')

In [2]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything()

In [3]:
train = pd.read_pickle('../Create_Features/preprocessed_train.pickle')
test = pd.read_pickle('../Create_Features/preprocessed_test.pickle')

In [4]:
n_trn = 415423
target_col = '18~20_ride'

In [5]:
train.head()

,18~20_ride,bus_route_id,in_out,latitude,longitude,station_code,dayofweek,weekend,ride_total,takeoff_total,ride_go_to_work,takeoff_go_to_work,dis_jejusi,dis_seoquipo,bus_route_id_station_code,bus_route_id_station_code_weekend,date_fq_enc,station_code_fq_enc,bus_route_id_fq_enc,bus_route_id_station_code_fq_enc,date_bus_route_id_fq_enc,date_station_code_fq_enc,date_bus_route_id_station_code_fq_enc,7~8_ride_date_mean,7~8_ride_date_bus_route_id_mean,8~9_ride_date_mean,8~9_ride_date_bus_route_id_mean,9~10_ride_date_mean,9~10_ride_date_bus_route_id_mean,station_sequence,station_reverse_sequence,weekday,is_national_holiday,getin_total,morning_getin,morning_takeoff,noon_getin,noon_takeoff,station_morning_getin_sum,station_morning_takeoff_sum,bus_route_getin_sum,bus_route_takeoff_sum,station_morning_getin_mean,station_morning_takeoff_mean,bus_route_getin_mean,bus_route_takeoff_mean,kmeans1,kmeans2,regular_commuter_count,afternoon_takeoff,next_bus_time_diff,getin_user_count1_morning,getin_user_count2_morning,takeoff_user_count1_noon,takeoff_user_count2_noon,hourly_rain,prev_daily_rain,hourly_cloud,latlong_second,total_population,man_population,woman_population,avg_time_diff,passengers_in,passengers_out,latitude_rank,longitude_rank
0,0.0,0,1,33.48990,126.49373,322,6,1,16.0,0.0,8.0,0.0,2.954920,26.256744,31053,53745,11538,46,1189,46,25,1,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,1,25,6,0,16.0,3.0,0.0,13.0,0.0,3.0,0.0,50.0,8.0,3.000000,0.000000,2.0,0.32,15,9,9.0,0.0,3650.490741,7.0,1.0,NaN,NaN,0.2,0.0,88,2411,43217.0,21189.0,22028.0,3113.015748,15.239130,1.909091,404772.5,214964.5
1,5.0,0,1,33.48944,126.48508,335,6,1,22.0,0.0,10.0,0.0,3.720275,26.403025,31054,53747,11538,2303,1189,46,25,45,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,2,24,6,0,22.0,9.0,0.0,13.0,0.0,183.0,116.0,50.0,8.0,4.066667,2.577778,2.0,0.32,15,9,466.0,0.0,3650.490741,197.0,NaN,95.0,NaN,0.2,0.0,88,2403,43217.0,21189.0,22028.0,1837.940774,594.000000,428.000000,396531.5,189710.0
2,2.0,0,1,33.48181,126.47352,408,6,1,4.0,0.0,3.0,0.0,5.036124,25.893305,31057,53753,11538,1154,1189,46,25,21,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,3,23,6,0,4.0,2.0,0.0,2.0,0.0,71.0,11.0,50.0,8.0,3.380952,0.523810,2.0,0.32,15,45,164.0,0.0,3650.490741,76.0,NaN,11.0,2.0,0.2,0.0,88,2347,56223.0,27761.0,28462.0,2448.248012,166.913043,28.000000,342909.5,156930.5
3,53.0,0,0,33.50577,126.49252,1448,6,1,79.0,0.0,49.0,0.0,2.864166,27.997494,31020,53682,11538,49,1189,46,25,1,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,4,22,6,0,79.0,23.0,0.0,56.0,0.0,23.0,0.0,50.0,8.0,23.000000,0.000000,2.0,0.32,4,3,2.0,0.0,3650.490741,49.0,NaN,NaN,NaN,0.2,0.0,88,2980,15673.0,7904.0,7769.0,3961.540412,89.130435,3.600000,518315.0,210660.0
4,0.0,0,0,33.25579,126.41260,1510,6,1,0.0,1.0,0.0,1.0,29.040353,13.574693,31022,53686,11538,386,1189,39,25,10,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,5,21,6,0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,50.0,8.0,0.100000,0.500000,2.0,0.32,85,136,3.0,0.0,3650.490741,4.0,NaN,5.0,NaN,0.2,0.0,88,562,4414.0,2252.0,2162.0,1058.012605,6.847826,27.456522,98750.5,93507.5


In [6]:
drop_cols = ['hourly_rain',
       'prev_daily_rain', 'hourly_cloud', 'latlong_second', 'total_population',
       'man_population', 'woman_population']

train= train.drop(drop_cols,1)
test= test.drop(drop_cols,1)


In [7]:
train.head()

,18~20_ride,bus_route_id,in_out,latitude,longitude,station_code,dayofweek,weekend,ride_total,takeoff_total,ride_go_to_work,takeoff_go_to_work,dis_jejusi,dis_seoquipo,bus_route_id_station_code,bus_route_id_station_code_weekend,date_fq_enc,station_code_fq_enc,bus_route_id_fq_enc,bus_route_id_station_code_fq_enc,date_bus_route_id_fq_enc,date_station_code_fq_enc,date_bus_route_id_station_code_fq_enc,7~8_ride_date_mean,7~8_ride_date_bus_route_id_mean,8~9_ride_date_mean,8~9_ride_date_bus_route_id_mean,9~10_ride_date_mean,9~10_ride_date_bus_route_id_mean,station_sequence,station_reverse_sequence,weekday,is_national_holiday,getin_total,morning_getin,morning_takeoff,noon_getin,noon_takeoff,station_morning_getin_sum,station_morning_takeoff_sum,bus_route_getin_sum,bus_route_takeoff_sum,station_morning_getin_mean,station_morning_takeoff_mean,bus_route_getin_mean,bus_route_takeoff_mean,kmeans1,kmeans2,regular_commuter_count,afternoon_takeoff,next_bus_time_diff,getin_user_count1_morning,getin_user_count2_morning,takeoff_user_count1_noon,takeoff_user_count2_noon,avg_time_diff,passengers_in,passengers_out,latitude_rank,longitude_rank
0,0.0,0,1,33.48990,126.49373,322,6,1,16.0,0.0,8.0,0.0,2.954920,26.256744,31053,53745,11538,46,1189,46,25,1,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,1,25,6,0,16.0,3.0,0.0,13.0,0.0,3.0,0.0,50.0,8.0,3.000000,0.000000,2.0,0.32,15,9,9.0,0.0,3650.490741,7.0,1.0,NaN,NaN,3113.015748,15.239130,1.909091,404772.5,214964.5
1,5.0,0,1,33.48944,126.48508,335,6,1,22.0,0.0,10.0,0.0,3.720275,26.403025,31054,53747,11538,2303,1189,46,25,45,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,2,24,6,0,22.0,9.0,0.0,13.0,0.0,183.0,116.0,50.0,8.0,4.066667,2.577778,2.0,0.32,15,9,466.0,0.0,3650.490741,197.0,NaN,95.0,NaN,1837.940774,594.000000,428.000000,396531.5,189710.0
2,2.0,0,1,33.48181,126.47352,408,6,1,4.0,0.0,3.0,0.0,5.036124,25.893305,31057,53753,11538,1154,1189,46,25,21,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,3,23,6,0,4.0,2.0,0.0,2.0,0.0,71.0,11.0,50.0,8.0,3.380952,0.523810,2.0,0.32,15,45,164.0,0.0,3650.490741,76.0,NaN,11.0,2.0,2448.248012,166.913043,28.000000,342909.5,156930.5
3,53.0,0,0,33.50577,126.49252,1448,6,1,79.0,0.0,49.0,0.0,2.864166,27.997494,31020,53682,11538,49,1189,46,25,1,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,4,22,6,0,79.0,23.0,0.0,56.0,0.0,23.0,0.0,50.0,8.0,23.000000,0.000000,2.0,0.32,4,3,2.0,0.0,3650.490741,49.0,NaN,NaN,NaN,3961.540412,89.130435,3.600000,518315.0,210660.0
4,0.0,0,0,33.25579,126.41260,1510,6,1,0.0,1.0,0.0,1.0,29.040353,13.574693,31022,53686,11538,386,1189,39,25,10,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,5,21,6,0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,50.0,8.0,0.100000,0.500000,2.0,0.32,85,136,3.0,0.0,3650.490741,4.0,NaN,5.0,NaN,1058.012605,6.847826,27.456522,98750.5,93507.5


In [8]:
# Before modeling
train_set = train.drop(target_col,1)
test_set = test.drop(target_col,1)

train_label = train[target_col]
test_label = test[target_col]

In [9]:
# Basic LGBM Model
n_splits= 5
NUM_BOOST_ROUND = 100000
SEED = 1993
lgbm_param = {'objective':'rmse',
              'boosting_type': 'gbdt',
              'random_state':1993,
              'learning_rate':0.05,
              'subsample':0.7,
              'tree_learner': 'serial',
              'colsample_bytree':0.78,
              'early_stopping_rounds':50,
              'subsample_freq': 1,
              'reg_lambda':7,
              'reg_alpha': 5,
              'num_leaves': 96,
              'seed' : SEED
            }

In [10]:
# StractifiedKfold & bus_route_id
split_col = 'bus_route_id'
len_seeds = 1

outer_stractified_busroute_oof_train = np.zeros( train.shape[0] )
outer_stractified_busroute_oof_test = np.zeros( test.shape[0] )

for _ in tqdm_notebook(range(len_seeds)):
    
    seed = random.randint(1, 100000)
    
    cv_list = []

    oof_train = np.zeros( train.shape[0] )
    final_test = np.zeros( test.shape[0] )

    kfolds = StratifiedKFold(n_splits = n_splits, shuffle=True, random_state=seed )

    for ind, (trn_ind, val_ind) in tqdm_notebook( enumerate(kfolds.split(train_set, train_set[split_col])) ):

        X_train, y_train = train_set.iloc[trn_ind], train_label[trn_ind]
        X_valid, y_valid = train_set.iloc[val_ind], train_label[val_ind]
        
        dtrain = lgbm.Dataset( X_train, y_train )
        dvalid = lgbm.Dataset( X_valid, y_valid ,reference=dtrain)
        
        lgbm_param['seed'] = seed

        model = lgbm.train(lgbm_param , dtrain, NUM_BOOST_ROUND, valid_sets=(dtrain, dvalid), valid_names=('train','valid'), 
                            categorical_feature=['bus_route_id','station_code','weekday',\
                                                                'kmeans1','kmeans2',
                                                                ] ,
                           verbose_eval= 100)

        valid_pred = model.predict(X_valid)
        test_pred  = model.predict(test_set)

        oof_train[val_ind] += valid_pred
        final_test += test_pred

        cv_list.append( sqrt(mean_squared_error(y_valid, valid_pred)) )

        print('='*80)

    final_test /= n_splits

    print(f"Average CV : {np.mean(cv_list)}")
    print(f"RMSE for OOF: {sqrt(mean_squared_error(train_label, oof_train))}")
    
    outer_stractified_busroute_oof_train += oof_train
    outer_stractified_busroute_oof_test += final_test
    
outer_stractified_busroute_oof_train /=len_seeds
outer_stractified_busroute_oof_test /= len_seeds

print(f"Overall for OOF: {sqrt(mean_squared_error(train_label, outer_stractified_busroute_oof_train))}")

# PostProcessing
outer_stractified_busroute_oof_train = [x if x>0 else 0 for x in  outer_stractified_busroute_oof_train]
outer_stractified_busroute_oof_test = [x if x>0 else 0 for x in  outer_stractified_busroute_oof_test]


print(f"RMSE for OOF: {sqrt(mean_squared_error(train_label, outer_stractified_busroute_oof_train))}")

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.87798	valid's rmse: 2.05096
[200]	train's rmse: 1.64976	valid's rmse: 2.0218
Early stopping, best iteration is:
[231]	train's rmse: 1.60414	valid's rmse: 2.01623
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.84434	valid's rmse: 2.31732
[200]	train's rmse: 1.62809	valid's rmse: 2.27343
[300]	train's rmse: 1.50146	valid's rmse: 2.26811
Early stopping, best iteration is:
[281]	train's rmse: 1.52221	valid's rmse: 2.26741
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.83417	valid's rmse: 2.38316
[200]	train's rmse: 1.61469	valid's rmse: 2.33944
[300]	train's rmse: 1.49065	valid's rmse: 2.33147
[400]	train's rmse: 1.40186	valid's rmse: 2.32681
[500]	train's rmse: 1.33241	valid's rmse: 2.31792
[600]	train's rmse: 1.27531	valid's rmse: 2.31544
Early stopping, best iteration is:
[604]	train's rmse: 1.27338	valid's rmse: 2.31529
Training unti

In [11]:
df_sub = pd.read_csv('../raw_dataset/submission_sample.csv')
df_sub['18~20_ride'] = outer_stractified_busroute_oof_test

df_sub.to_csv('../submission/after.csv',index=False)

In [12]:
# StractifiedKfold & bus_route_id
split_col = 'bus_route_id'
len_seeds = 40

outer_stractified_busroute_oof_train = np.zeros( train.shape[0] )
outer_stractified_busroute_oof_test = np.zeros( test.shape[0] )

for _ in tqdm_notebook(range(len_seeds)):
    
    seed = random.randint(1, 100000)
    
    cv_list = []

    oof_train = np.zeros( train.shape[0] )
    final_test = np.zeros( test.shape[0] )

    kfolds = StratifiedKFold(n_splits = n_splits, shuffle=True, random_state=seed )

    for ind, (trn_ind, val_ind) in tqdm_notebook( enumerate(kfolds.split(train_set, train_set[split_col])) ):

        X_train, y_train = train_set.iloc[trn_ind], train_label[trn_ind]
        X_valid, y_valid = train_set.iloc[val_ind], train_label[val_ind]
        
        dtrain = lgbm.Dataset( X_train, y_train )
        dvalid = lgbm.Dataset( X_valid, y_valid ,reference=dtrain)
        
        lgbm_param['seed'] = seed

        model = lgbm.train(lgbm_param , dtrain, NUM_BOOST_ROUND, valid_sets=(dtrain, dvalid), valid_names=('train','valid'), 
                            categorical_feature=['bus_route_id','station_code','weekday',\
                                                                'kmeans1','kmeans2',
                                                                ] ,
                           verbose_eval= 100)

        valid_pred = model.predict(X_valid)
        test_pred  = model.predict(test_set)

        oof_train[val_ind] += valid_pred
        final_test += test_pred

        cv_list.append( sqrt(mean_squared_error(y_valid, valid_pred)) )

        print('='*80)

    final_test /= n_splits

    print(f"Average CV : {np.mean(cv_list)}")
    print(f"RMSE for OOF: {sqrt(mean_squared_error(train_label, oof_train))}")
    
    outer_stractified_busroute_oof_train += oof_train
    outer_stractified_busroute_oof_test += final_test
    
outer_stractified_busroute_oof_train /=len_seeds
outer_stractified_busroute_oof_test /= len_seeds

print(f"Overall for OOF: {sqrt(mean_squared_error(train_label, outer_stractified_busroute_oof_train))}")

# PostProcessing
outer_stractified_busroute_oof_train = [x if x>0 else 0 for x in  outer_stractified_busroute_oof_train]
outer_stractified_busroute_oof_test = [x if x>0 else 0 for x in  outer_stractified_busroute_oof_test]


print(f"RMSE for OOF: {sqrt(mean_squared_error(train_label, outer_stractified_busroute_oof_train))}")

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.83657	valid's rmse: 2.23956
[200]	train's rmse: 1.61828	valid's rmse: 2.22512
Early stopping, best iteration is:
[200]	train's rmse: 1.61828	valid's rmse: 2.22512
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.86582	valid's rmse: 2.14657
[200]	train's rmse: 1.64402	valid's rmse: 2.1241
[300]	train's rmse: 1.51256	valid's rmse: 2.11787
[400]	train's rmse: 1.4202	valid's rmse: 2.11268
[500]	train's rmse: 1.34762	valid's rmse: 2.111
Early stopping, best iteration is:
[503]	train's rmse: 1.34545	valid's rmse: 2.11054
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.86359	valid's rmse: 2.18658
[200]	train's rmse: 1.63914	valid's rmse: 2.14577
[300]	train's rmse: 1.50899	valid's rmse: 2.13736
[400]	train's rmse: 1.41446	valid's rmse: 2.12998
Early stopping, best iteration is:
[439]	train's rmse: 1.38426	valid's rmse: 2.12734
Training until v

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.87028	valid's rmse: 2.17168
[200]	train's rmse: 1.64828	valid's rmse: 2.14186
[300]	train's rmse: 1.51719	valid's rmse: 2.12708
[400]	train's rmse: 1.42267	valid's rmse: 2.12151
[500]	train's rmse: 1.34918	valid's rmse: 2.11837
Early stopping, best iteration is:
[516]	train's rmse: 1.33882	valid's rmse: 2.11689
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.84528	valid's rmse: 2.25942
[200]	train's rmse: 1.62479	valid's rmse: 2.21655
[300]	train's rmse: 1.49813	valid's rmse: 2.20859
[400]	train's rmse: 1.406	valid's rmse: 2.20361
[500]	train's rmse: 1.33524	valid's rmse: 2.20127
[600]	train's rmse: 1.27789	valid's rmse: 2.20055
[700]	train's rmse: 1.23081	valid's rmse: 2.19818
[800]	train's rmse: 1.19028	valid's rmse: 2.19753
Early stopping, best iteration is:
[754]	train's rmse: 1.20804	valid's rmse: 2.1972
Training until validation scores don't improve for 50 rounds


Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.85864	valid's rmse: 2.19229
[200]	train's rmse: 1.63846	valid's rmse: 2.17174
Early stopping, best iteration is:
[192]	train's rmse: 1.65096	valid's rmse: 2.17131
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.86931	valid's rmse: 2.14892
[200]	train's rmse: 1.64349	valid's rmse: 2.118
[300]	train's rmse: 1.51295	valid's rmse: 2.11138
[400]	train's rmse: 1.41737	valid's rmse: 2.10704
Early stopping, best iteration is:
[362]	train's rmse: 1.45023	valid's rmse: 2.10545
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.87148	valid's rmse: 2.15865
[200]	train's rmse: 1.64609	valid's rmse: 2.113
[300]	train's rmse: 1.51514	valid's rmse: 2.0984
[400]	train's rmse: 1.42161	valid's rmse: 2.08962
[500]	train's rmse: 1.34848	valid's rmse: 2.08731
[600]	train's rmse: 1.29099	valid's rmse: 2.08443
Early stopping, best iteration is:
[576]	train's rms

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.86762	valid's rmse: 2.07533
[200]	train's rmse: 1.64115	valid's rmse: 2.04745
[300]	train's rmse: 1.50959	valid's rmse: 2.04036
[400]	train's rmse: 1.41717	valid's rmse: 2.035
Early stopping, best iteration is:
[355]	train's rmse: 1.45559	valid's rmse: 2.03468
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.86263	valid's rmse: 2.15979
[200]	train's rmse: 1.63679	valid's rmse: 2.12476
[300]	train's rmse: 1.50392	valid's rmse: 2.10617
[400]	train's rmse: 1.41123	valid's rmse: 2.10058
[500]	train's rmse: 1.33952	valid's rmse: 2.09524
[600]	train's rmse: 1.28077	valid's rmse: 2.09202
[700]	train's rmse: 1.23261	valid's rmse: 2.08856
Early stopping, best iteration is:
[744]	train's rmse: 1.21363	valid's rmse: 2.08818
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.86026	valid's rmse: 2.1877
[200]	train's rmse: 1.63486	valid's rmse: 2.156
[3

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.85429	valid's rmse: 2.23496
[200]	train's rmse: 1.63185	valid's rmse: 2.20196
[300]	train's rmse: 1.50203	valid's rmse: 2.18558
[400]	train's rmse: 1.41014	valid's rmse: 2.17728
[500]	train's rmse: 1.34014	valid's rmse: 2.1758
[600]	train's rmse: 1.28165	valid's rmse: 2.1744
[700]	train's rmse: 1.23347	valid's rmse: 2.17367
Early stopping, best iteration is:
[664]	train's rmse: 1.2503	valid's rmse: 2.17267
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.83485	valid's rmse: 2.3356
[200]	train's rmse: 1.61857	valid's rmse: 2.29931
[300]	train's rmse: 1.49012	valid's rmse: 2.28662
[400]	train's rmse: 1.39817	valid's rmse: 2.28014
[500]	train's rmse: 1.32868	valid's rmse: 2.276
Early stopping, best iteration is:
[506]	train's rmse: 1.32502	valid's rmse: 2.2757
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.87067	valid's rmse: 2.1596
[200]

...

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.8523	valid's rmse: 2.29219
[200]	train's rmse: 1.63136	valid's rmse: 2.23633
[300]	train's rmse: 1.50338	valid's rmse: 2.22998
Early stopping, best iteration is:
[324]	train's rmse: 1.47813	valid's rmse: 2.22871
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.86744	valid's rmse: 2.09595
[200]	train's rmse: 1.6467	valid's rmse: 2.07708
[300]	train's rmse: 1.51388	valid's rmse: 2.06675
Early stopping, best iteration is:
[331]	train's rmse: 1.48131	valid's rmse: 2.0647
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.87129	valid's rmse: 2.06437
[200]	train's rmse: 1.64041	valid's rmse: 2.05229
Early stopping, best iteration is:
[154]	train's rmse: 1.72748	valid's rmse: 2.04874
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.84372	valid's rmse: 2.28655
[200]	train's rmse: 1.62992	valid's rmse: 2.25071
[300

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.8496	valid's rmse: 2.20367
[200]	train's rmse: 1.62568	valid's rmse: 2.18581
[300]	train's rmse: 1.49694	valid's rmse: 2.18512
[400]	train's rmse: 1.40522	valid's rmse: 2.18352
Early stopping, best iteration is:
[368]	train's rmse: 1.43211	valid's rmse: 2.18204
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.84983	valid's rmse: 2.28905
[200]	train's rmse: 1.63124	valid's rmse: 2.23679
[300]	train's rmse: 1.50154	valid's rmse: 2.22908
[400]	train's rmse: 1.41106	valid's rmse: 2.21983
[500]	train's rmse: 1.33897	valid's rmse: 2.21199
[600]	train's rmse: 1.28202	valid's rmse: 2.20827
Early stopping, best iteration is:
[589]	train's rmse: 1.28778	valid's rmse: 2.20724
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.84485	valid's rmse: 2.32901
[200]	train's rmse: 1.62661	valid's rmse: 2.28533
[300]	train's rmse: 1.49754	valid's rmse: 2.2811

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.83249	valid's rmse: 2.37617
[200]	train's rmse: 1.619	valid's rmse: 2.32409
[300]	train's rmse: 1.49265	valid's rmse: 2.30663
Early stopping, best iteration is:
[312]	train's rmse: 1.4807	valid's rmse: 2.30535
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.85403	valid's rmse: 2.17209
[200]	train's rmse: 1.63081	valid's rmse: 2.16428
Early stopping, best iteration is:
[210]	train's rmse: 1.61521	valid's rmse: 2.16263
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.88532	valid's rmse: 2.15956
[200]	train's rmse: 1.65957	valid's rmse: 2.09603
[300]	train's rmse: 1.52645	valid's rmse: 2.07465
[400]	train's rmse: 1.43027	valid's rmse: 2.06601
[500]	train's rmse: 1.3567	valid's rmse: 2.06195
[600]	train's rmse: 1.29687	valid's rmse: 2.05503
Early stopping, best iteration is:
[603]	train's rmse: 1.29522	valid's rmse: 2.05451
Training until v

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.86615	valid's rmse: 2.12054
[200]	train's rmse: 1.64436	valid's rmse: 2.07878
[300]	train's rmse: 1.51254	valid's rmse: 2.07254
[400]	train's rmse: 1.42034	valid's rmse: 2.06968
Early stopping, best iteration is:
[378]	train's rmse: 1.43878	valid's rmse: 2.0687
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.8602	valid's rmse: 2.28038
[200]	train's rmse: 1.6373	valid's rmse: 2.23244
[300]	train's rmse: 1.50652	valid's rmse: 2.22393
[400]	train's rmse: 1.41317	valid's rmse: 2.21857
[500]	train's rmse: 1.34087	valid's rmse: 2.21145
[600]	train's rmse: 1.28219	valid's rmse: 2.20911
Early stopping, best iteration is:
[559]	train's rmse: 1.30515	valid's rmse: 2.20869
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.84711	valid's rmse: 2.35927
[200]	train's rmse: 1.6254	valid's rmse: 2.31839
[300]	train's rmse: 1.49526	valid's rmse: 2.30935
[

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.85686	valid's rmse: 2.17175
[200]	train's rmse: 1.63613	valid's rmse: 2.15395
[300]	train's rmse: 1.5062	valid's rmse: 2.14474
[400]	train's rmse: 1.41473	valid's rmse: 2.1387
[500]	train's rmse: 1.3428	valid's rmse: 2.13622
[600]	train's rmse: 1.28536	valid's rmse: 2.13346
[700]	train's rmse: 1.23586	valid's rmse: 2.13278
Early stopping, best iteration is:
[654]	train's rmse: 1.25823	valid's rmse: 2.13087
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 1.85554	valid's rmse: 2.22456
[200]	train's rmse: 1.63956	valid's rmse: 2.19153
[300]	train's rmse: 1.50801	valid's rmse: 2.18189
[400]	train's rmse: 1.41601	valid's rmse: 2.17629
[500]	train's rmse: 1.3449	valid's rmse: 2.1748
[600]	train's rmse: 1.28686	valid's rmse: 2.17363
Early stopping, best iteration is:
[587]	train's rmse: 1.29411	valid's rmse: 2.17291
Training until validation scores don't improve for 50 rounds
[1

In [14]:
df_oof = pd.read_csv('../raw_dataset/train.csv', usecols = ['id','18~20_ride'])
df_oof['18~20_ride'] = outer_stractified_busroute_oof_train

# df_oof.to_csv('../oof/lgbm_40_seeds_stractified5k_bus_route_id.csv',index=False)
df_oof.to_csv('../lgbm_40_seeds_stractified5k_bus_route_id.csv',index=False)

df_sub = pd.read_csv('../raw_dataset/submission_sample.csv')
df_sub['18~20_ride'] = outer_stractified_busroute_oof_test

df_sub.to_csv('../submission/lgbm_40_seeds_stractified5k_bus_route_id.csv',index=False)